# Sentiment analysis
The following notebook includes experiments related to BERT, RoBERTa and ensemble of models. Those experiments were conducted on Google Colab using available GPU where possible.

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.metrics import balanced_accuracy_score
import numpy as np
import torch
import time
from transformers import BertForSequenceClassification, BertTokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
path_to_dir="" #to reproduce the results, it is necessary to change this into suitable path
where_to_save="/results/" #to reproduce the results, it is necessary to change this into suitable path

In [ ]:
with open(path_to_dir+"chosen_articles.txt", "r") as f:
    not_preprocessed_data = f.readlines()

with open(path_to_dir+"chosen_articles_cleaned_4o.txt", "r") as f:
    processed_4o_data = f.readlines()

with open(path_to_dir+"chosen_articles_cleaned_regex.txt", "r") as f:
    processed_regex_data = f.readlines()

with open(path_to_dir+"chosen_articles_cleaned_by_me.txt", "r") as f:
    processed_human_data = f.readlines()

df_my=pd.read_csv(path_to_dir+"articles_categories_my_gt_2.csv", sep=';')

In [ ]:
test_tse_data=pd.read_csv(path_to_dir+"tweet-sentiment-extraction/test.csv")

In [ ]:
test_ns_data2=pd.read_csv(path_to_dir+"news_sentiment_dataset/Sentiment_dataset.csv")
test_ns_data2

,news_title,reddit_title,sentiment,text,url
0,Mark Cuban launches generic drug company,Billionaire Mark Cuban just launched a drug co...,1.0,Billionaire investor and Shark Tank star Mark ...,https://www.beckershospitalreview.com/pharmacy...
1,From Defendant to Defender: One Wrongfully Con...,"Man falsely imprisoned for 10 years, uses pris...",1.0,Attorney Jarrett Adams recently helped overtur...,https://www.nbcnews.com/news/us-news/defendant...
2,"Amazon Tribe Wins Lawsuit Against Big Oil, Sav...",Amazon tribe wins legal battle against oil com...,1.0,The Amazon Rainforest is well known across the...,https://www.disclose.tv/amazon-tribe-wins-laws...
3,Newark police: No officer fired a single shot ...,Newark police: No officer fired a single shot ...,1.0,Newark police: No officer fired a single shot ...,https://newjersey.news12.com/newark-police-no-...
4,Ingen barn døde i trafikken i 2019,No children died in traffic accidents in Norwa...,1.0,I 1970 døde det 560 mennesker i den norske tra...,https://www.nrk.no/trondelag/ingen-barn-dode-i...
...,...,...,...,...,...
843,Dee Why attack: Man allegedly choked and threa...,Dee Why attack: Man allegedly choked and threa...,0.0,Frightening details have emerged about a toile...,https://www.9news.com.au/2018/11/30/17/55/sydn...
844,Africa: Children and HIV/Aids - 'We Need to Ta...,Africa: Children and HIV/Aids - 'We Need to Ta...,0.0,"interview\n\nJohannesburg — 360,000 adolescent...",https://allafrica.com/stories/201811300567.html
845,Terrorism suspected in Eilat attack,Terrorism suspected in Eilat attack,0.0,A violent attack in the southern Israeli port ...,http://www.israelnationalnews.com/News/News.as...
846,Anti-Semitism never disappeared in Europe. It'...,Anti-Semitism never disappeared in Europe. It'...,0.0,"It's a 17-year-old boy, too frightened to wear...",https://edition.cnn.com/2018/11/27/europe/anti...


In [ ]:
actaware_df=pd.read_csv(path_to_dir+"actaware_df_with_sentiment_graph.csv")
actaware_df=actaware_df.assign(Sentiment_BERT='', Sentiment_RoBERTa='')

In [ ]:
#results from graph-based
graph_results_by_me=pd.read_csv(path_to_dir+"actaware_df_with_sentiment_graph_chosen_articles_cleaned_by_me.csv", sep=',')

graph_results_by_4o=pd.read_csv(path_to_dir+"actaware_df_with_sentiment_graph_chosen_articles_cleaned_4o.csv", sep=',')

In [ ]:
def run_bert_sentiment(list_of_contents):
  sentiment_list=[" " for i in range(len(list_of_contents))]
  model = BertForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
  tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
  device = "cuda:0" if torch.cuda.is_available() else "cpu"
  i=0
  while i < len(list_of_contents):
    if i%200==0:
      print(i)
    input_ids = tokenizer.encode(list_of_contents[i], return_tensors="pt", max_length=512, truncation=True)
    input_ids = input_ids.to(device)
    model = model.to(device)
    output=model(input_ids)
    predictions = output.logits.argmax(dim=1)
    sentiment_lab=['1 star', '2 stars', '3 stars', '4 stars', '5 stars']
    sentiment_list[i]=sentiment_lab[predictions.item()]
    i+=1
  return sentiment_list

In [ ]:
def run_bert_sentiment_my_gt(list_of_contents):
  sentiment_list=[" " for i in range(len(list_of_contents))]
  model = BertForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
  tokenizer = BertTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
  device = "cuda:0" if torch.cuda.is_available() else "cpu"

  i=0
  while i < len(list_of_contents):
    input_ids = tokenizer.encode(list_of_contents[i], return_tensors="pt", max_length=512, truncation=True)
    input_ids = input_ids.to(device)
    model = model.to(device)
    output=model(input_ids)
    predictions = output.logits.argmax(dim=1)
    sentiment_lab=['negative', 'negative', 'neutral', 'positive', 'positive']
    sentiment_list[i]=sentiment_lab[predictions.item()]
    i+=1
  return sentiment_list

In [ ]:
def run_roberta_sentiment(list_of_contents):
  sentiment_list=[" " for i in range(len(list_of_contents))]
  model_name="textattack/roberta-base-SST-2"
  model = AutoModelForSequenceClassification.from_pretrained(model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  device = "cuda:0" if torch.cuda.is_available() else "cpu"

  i=0
  while i < len(list_of_contents):
    if i%200==0:
      print(i)
    input_ids = tokenizer.encode(list_of_contents[i], return_tensors="pt", max_length=512, truncation=True)
    input_ids = input_ids.to(device)
    model = model.to(device)
    output=model(input_ids)
    predictions = output.logits.argmax().item()
    sentiment_lab=['negative', 'positive']
    sentiment_list[i]=sentiment_lab[predictions]
    i+=1
  return sentiment_list

## Experiments on subset of Actaware

In [ ]:
for i in range(3):
  start_time=time.time()
  results=run_bert_sentiment_my_gt(not_preprocessed_data)
  score_result=balanced_accuracy_score(df_my['Sentiment'], results)
  print(f"Time: {time.time()-start_time}")
  print(f"Balanced accuracy for not processed: {score_result}")

for i in range(3):
  start_time=time.time()
  results=run_bert_sentiment_my_gt(processed_4o_data)
  score_result=balanced_accuracy_score(df_my['Sentiment'], results)
  print(f"Time: {time.time()-start_time}")
  print(f"Balanced accuracy for processed 4o: {score_result}")

for i in range(3):
  start_time=time.time()
  results=run_bert_sentiment_my_gt(processed_regex_data)
  score_result=balanced_accuracy_score(df_my['Sentiment'], results)
  print(f"Time: {time.time()-start_time}")
  print(f"Balanced accuracy for processed regex: {score_result}")

for i in range(3):
  start_time=time.time()
  results=run_bert_sentiment_my_gt(processed_human_data)
  score_result=balanced_accuracy_score(df_my['Sentiment'], results)
  print(f"Time: {time.time()-start_time}")
  print(f"Balanced accuracy for processed by human: {score_result}")

Time: 7.95191216468811
Balanced accuracy for not processed: 0.4911673294026235
Time: 8.373095512390137
Balanced accuracy for not processed: 0.4911673294026235
Time: 9.010267734527588
Balanced accuracy for not processed: 0.4911673294026235
Time: 11.170091390609741
Balanced accuracy for processed 4o: 0.4911673294026235
Time: 6.139367341995239
Balanced accuracy for processed 4o: 0.4911673294026235
Time: 4.93052339553833
Balanced accuracy for processed 4o: 0.4911673294026235
Time: 5.489236831665039
Balanced accuracy for processed regex: 0.5107751725398785
Time: 7.198122978210449
Balanced accuracy for processed regex: 0.5107751725398785
Time: 7.943602561950684
Balanced accuracy for processed regex: 0.5107751725398785
Time: 10.399333000183105
Balanced accuracy for processed by human: 0.5107751725398785
Time: 8.720696687698364
Balanced accuracy for processed by human: 0.5107751725398785
Time: 5.719872951507568
Balanced accuracy for processed by human: 0.5107751725398785


In [ ]:
for i in range(3):
  start_time=time.time()
  results=run_roberta_sentiment(not_preprocessed_data)
  score_result=balanced_accuracy_score(df_my['Sentiment'], results)
  print(f"Time: {time.time()-start_time}")
  print(f"Balanced accuracy for RoBERTa not processed: {score_result}")

for i in range(3):
  start_time=time.time()
  results=run_roberta_sentiment(processed_4o_data)
  score_result=balanced_accuracy_score(df_my['Sentiment'], results)
  print(f"Time: {time.time()-start_time}")
  print(f"Balanced accuracy for RoBERTa for processed 4o: {score_result}")

for i in range(3):
  start_time=time.time()
  results=run_roberta_sentiment(processed_regex_data)
  score_result=balanced_accuracy_score(df_my['Sentiment'], results)
  print(f"Time: {time.time()-start_time}")
  print(f"Balanced accuracy for RoBERTa for processed regex: {score_result}")

for i in range(3):
  start_time=time.time()
  results=run_roberta_sentiment(processed_human_data)
  score_result=balanced_accuracy_score(df_my['Sentiment'], results)
  print(f"Time: {time.time()-start_time}")
  print(f"Balanced accuracy for RoBERTa for processed by human: {score_result}")

config.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at textattack/roberta-base-SST-2 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

0
Time: 4.203669548034668
Balanced accuracy for RoBERTa for processed by human: 0.6122994652406417


Some weights of the model checkpoint at textattack/roberta-base-SST-2 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0
Time: 3.9347774982452393
Balanced accuracy for RoBERTa for processed by human: 0.6122994652406417


Some weights of the model checkpoint at textattack/roberta-base-SST-2 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0
Time: 3.940716505050659
Balanced accuracy for RoBERTa for processed by human: 0.6122994652406417


## Twitter sentiment dataset

### BERT

In [ ]:
test_tse_data["predicted_sentiment_bert"]=''

In [ ]:
tse_texts_list=list(test_tse_data['text'])

In [ ]:
tse_sentiment_list=run_bert_sentiment(tse_texts_list)

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400


In [ ]:
test_tse_data['predicted_sentiment_bert']=tse_sentiment_list
test_tse_data

,textID,text,sentiment,predicted_sentiment_bert
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,positive
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,negative
3,01082688c6,happy bday!,positive,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,positive
...,...,...,...,...
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",negative,neutral
3530,416863ce47,All alone in this old house again. Thanks for...,positive,positive
3531,6332da480c,I know what you mean. My little dog is sinkin...,negative,negative
3532,df1baec676,_sutra what is your next youtube video gonna b...,positive,positive


In [ ]:
accuracy=np.sum(test_tse_data['sentiment']==test_tse_data['predicted_sentiment_bert'])/test_tse_data.shape[0]

In [ ]:
ba_tse=balanced_accuracy_score(test_tse_data['sentiment'], test_tse_data['predicted_sentiment_bert'])

In [ ]:
accuracy, ba_tse

(0.5325410299943407, 0.5755031610879299)

### RoBERTa

In [ ]:
tse_sentiment_list_roberta=run_roberta_sentiment(tse_texts_list)

Some weights of the model checkpoint at textattack/roberta-base-SST-2 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400


In [ ]:
accuracy_roberta=np.sum(test_tse_data['sentiment']==tse_sentiment_list_roberta)/test_tse_data.shape[0]

In [ ]:
ba_tse_roberta=balanced_accuracy_score(test_tse_data['sentiment'], tse_sentiment_list_roberta)

In [ ]:
accuracy_roberta, ba_tse_roberta

(0.5181097906055461, 0.580623063850625)

In [ ]:
test_tse_data['predicted_sentiment_roberta'] = tse_sentiment_list_roberta
test_tse_data.head()

,textID,text,sentiment,predicted_sentiment_bert,predicted_sentiment_roberta
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,positive,positive
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,positive,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,negative,negative
3,01082688c6,happy bday!,positive,positive,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,positive,positive


In [ ]:
test_tse_data.to_csv(where_to_save+"test_tse_data_bert_roberta.csv", index=False)

## New sentiment dataset

### BERT

In [ ]:
def map_values(x):
  if x==1:
    return 'positive'
  elif x==0:
    return 'negative'
  else:
    return 'neutral'

test_ns_data2['sentiment']=test_ns_data2['sentiment'].apply(map_values)

In [ ]:
def cut_to_bert(string_list, max_words=300):
  cut_strings=["" for _ in string_list]
  for i, string in enumerate(string_list):
    words=string.split()[:max_words]
    joined=' '.join(words)
    cut_strings[i]=joined
  return cut_strings

In [ ]:
test_ns_data2["predicted_sentiment_bert"]=''

In [ ]:
ns_texts_list=test_ns_data2['text']
ns_sentiment_list=run_bert_sentiment(ns_texts_list)

0
200
400
600
800


In [ ]:
sum=0
for i in range(len(list(test_ns_data2['sentiment']))):
  if test_ns_data2['sentiment'][i]==ns_sentiment_list[i]:
    sum+=1

In [ ]:
accuracy_ns=sum/test_ns_data2.shape[0]

In [ ]:
ba_ns=balanced_accuracy_score(list(test_ns_data2['sentiment']), ns_sentiment_list)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [ ]:
accuracy_ns, ba_ns

(0.6084905660377359, 0.6481283422459894)

### RoBERTa

In [ ]:
ns_sentiment_list_roberta=run_roberta_sentiment(ns_texts_list)

Some weights of the model checkpoint at textattack/roberta-base-SST-2 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0
200
400
600
800


In [ ]:
sum_roberta=0
for i in range(len(list(test_ns_data2['sentiment']))):
  if test_ns_data2['sentiment'][i]==ns_sentiment_list_roberta[i]:
    sum_roberta+=1

In [ ]:
accuracy_ns_roberta=sum_roberta/test_ns_data2.shape[0]

In [ ]:
ba_ns_roberta=balanced_accuracy_score(list(test_ns_data2['sentiment']), ns_sentiment_list_roberta)

In [ ]:
accuracy_ns_roberta, ba_ns_roberta

(0.7900943396226415, 0.7770588235294118)

In [ ]:
test_ns_data2['predicted_sentiment_bert'] = ns_sentiment_list
test_ns_data2['predicted_sentiment_roberta'] = ns_sentiment_list_roberta
test_ns_data2.head()

,news_title,reddit_title,sentiment,text,url,predicted_sentiment_bert,predicted_sentiment_roberta
0,Mark Cuban launches generic drug company,Billionaire Mark Cuban just launched a drug co...,positive,Billionaire investor and Shark Tank star Mark ...,https://www.beckershospitalreview.com/pharmacy...,negative,positive
1,From Defendant to Defender: One Wrongfully Con...,"Man falsely imprisoned for 10 years, uses pris...",positive,Attorney Jarrett Adams recently helped overtur...,https://www.nbcnews.com/news/us-news/defendant...,positive,negative
2,"Amazon Tribe Wins Lawsuit Against Big Oil, Sav...",Amazon tribe wins legal battle against oil com...,positive,The Amazon Rainforest is well known across the...,https://www.disclose.tv/amazon-tribe-wins-laws...,positive,positive
3,Newark police: No officer fired a single shot ...,Newark police: No officer fired a single shot ...,positive,Newark police: No officer fired a single shot ...,https://newjersey.news12.com/newark-police-no-...,negative,positive
4,Ingen barn døde i trafikken i 2019,No children died in traffic accidents in Norwa...,positive,I 1970 døde det 560 mennesker i den norske tra...,https://www.nrk.no/trondelag/ingen-barn-dode-i...,negative,positive


In [ ]:
test_ns_data2.to_csv(where_to_save+"test_ns_data2_bert_roberta.csv", index=False)

In [ ]:
np.sum(test_ns_data2['Sentiment']==test_ns_data2['Sentiment_RoBERTa'])/test_ns_data2.shape[0]

## Actaware dataset

In [ ]:
texts_actaware=actaware_df['Text']
print("starting BERT")
actaware_df['Sentiment_BERT']=run_bert_sentiment(texts_actaware)
print("starting RoBERTa")
actaware_df['Sentiment_RoBERTa']=run_roberta_sentiment(texts_actaware)

starting BERT
0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
starting RoBERTa


config.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at textattack/roberta-base-SST-2 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600


In [ ]:
actaware_df.head()

,Text,Sentiment,Sentiment_BERT,Sentiment_RoBERTa
0,The BBC is being urged to drop singer Olly Ale...,negative,negative,negative
1,"Olly Alexander, the UK's new Eurovision act, h...",negative,negative,negative
2,A union representative claims they were barred...,negative,negative,negative
3,The Duke invited a gun smuggler and an alleged...,negative,negative,negative
4,Airbnb is set to use artificial intelligence t...,positive,positive,negative


In [ ]:
actaware_df.to_csv(where_to_save+"actaware_df_with_sentiment_graph_bert_roberta.csv", index=False)

In [ ]:
actaware_df.shape

(10698, 4)

In [ ]:
np.sum(actaware_df['Sentiment']==actaware_df['Sentiment_RoBERTa'])/actaware_df.shape[0] #how many of the predictions are the same for BERT and RoBERTa

0.6505888951205833

## Comparison between graph and bert and roberta for other datasets

In [ ]:
test_tse_data,test_ns_data2

In [ ]:
test_tse_data_graph=pd.read_csv(path_to_dir+"test_tse_data_graph.csv")
test_ns_data2_graph=pd.read_csv(path_to_dir+"test_ns_data2.csv")

In [ ]:
test_tse_data_graph.head()

,textID,text,sentiment,predicted_sentiment,predicted_sentiment_compound
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,neutral,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,positive,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,negative,negative
3,01082688c6,happy bday!,positive,neutral,neutral
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,positive,positive


In [ ]:
test_tse_data.head()

,textID,text,sentiment,predicted_sentiment_bert,predicted_sentiment_roberta
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,positive,positive
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,positive,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,negative,negative
3,01082688c6,happy bday!,positive,positive,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,positive,positive


In [ ]:
#graph compound False vs compound true

np.sum(test_tse_data_graph['predicted_sentiment']==test_tse_data_graph['predicted_sentiment_compound'])/test_tse_data_graph.shape[0]

0.9275608375778155

In [ ]:
#graph compound False vs bert

np.sum(test_tse_data_graph['predicted_sentiment']==test_tse_data['predicted_sentiment_bert'])/test_tse_data_graph.shape[0]

0.4128466327108093

In [ ]:
#graph compound False vs roberta

np.sum(test_tse_data_graph['predicted_sentiment']==test_tse_data['predicted_sentiment_roberta'])/test_tse_data_graph.shape[0]

0.4020939445387663

In [ ]:
#graph compound true vs bert

np.sum(test_tse_data_graph['predicted_sentiment_compound']==test_tse_data['predicted_sentiment_bert'])/test_tse_data_graph.shape[0]

0.3992642897566497

In [ ]:
#graph compound False vs roberta

np.sum(test_tse_data_graph['predicted_sentiment_compound']==test_tse_data['predicted_sentiment_roberta'])/test_tse_data_graph.shape[0]

0.3754951895868704

In [ ]:
#bert vs roberta

np.sum(test_tse_data['predicted_sentiment_bert']==test_tse_data['predicted_sentiment_roberta'])/test_tse_data.shape[0]

0.6994906621392191

news

In [ ]:
test_ns_data2.head()

,news_title,reddit_title,sentiment,text,url,predicted_sentiment_bert,predicted_sentiment_roberta
0,Mark Cuban launches generic drug company,Billionaire Mark Cuban just launched a drug co...,positive,Billionaire investor and Shark Tank star Mark ...,https://www.beckershospitalreview.com/pharmacy...,negative,positive
1,From Defendant to Defender: One Wrongfully Con...,"Man falsely imprisoned for 10 years, uses pris...",positive,Attorney Jarrett Adams recently helped overtur...,https://www.nbcnews.com/news/us-news/defendant...,positive,negative
2,"Amazon Tribe Wins Lawsuit Against Big Oil, Sav...",Amazon tribe wins legal battle against oil com...,positive,The Amazon Rainforest is well known across the...,https://www.disclose.tv/amazon-tribe-wins-laws...,positive,positive
3,Newark police: No officer fired a single shot ...,Newark police: No officer fired a single shot ...,positive,Newark police: No officer fired a single shot ...,https://newjersey.news12.com/newark-police-no-...,negative,positive
4,Ingen barn døde i trafikken i 2019,No children died in traffic accidents in Norwa...,positive,I 1970 døde det 560 mennesker i den norske tra...,https://www.nrk.no/trondelag/ingen-barn-dode-i...,negative,positive


In [ ]:
test_ns_data2_graph.head()

,news_title,reddit_title,sentiment,text,url,predicted_sentiment,predicted_sentiment_compound
0,Mark Cuban launches generic drug company,Billionaire Mark Cuban just launched a drug co...,positive,Billionaire investor and Shark Tank star Mark ...,https://www.beckershospitalreview.com/pharmacy...,positive,positive
1,From Defendant to Defender: One Wrongfully Con...,"Man falsely imprisoned for 10 years, uses pris...",positive,Attorney Jarrett Adams recently helped overtur...,https://www.nbcnews.com/news/us-news/defendant...,negative,positive
2,"Amazon Tribe Wins Lawsuit Against Big Oil, Sav...",Amazon tribe wins legal battle against oil com...,positive,The Amazon Rainforest is well known across the...,https://www.disclose.tv/amazon-tribe-wins-laws...,positive,positive
3,Newark police: No officer fired a single shot ...,Newark police: No officer fired a single shot ...,positive,Newark police: No officer fired a single shot ...,https://newjersey.news12.com/newark-police-no-...,negative,negative
4,Ingen barn døde i trafikken i 2019,No children died in traffic accidents in Norwa...,positive,I 1970 døde det 560 mennesker i den norske tra...,https://www.nrk.no/trondelag/ingen-barn-dode-i...,positive,positive


In [ ]:
#graph compound true vs compound false

np.sum(test_ns_data2_graph['predicted_sentiment_compound']==test_ns_data2_graph['predicted_sentiment'])/test_ns_data2.shape[0]

0.9245283018867925

In [ ]:
#graph compound False vs bert

np.sum(test_ns_data2_graph['predicted_sentiment']==test_ns_data2['predicted_sentiment_bert'])/test_ns_data2.shape[0]

0.6415094339622641

In [ ]:
#graph compound False vs roberta

np.sum(test_ns_data2_graph['predicted_sentiment']==test_ns_data2['predicted_sentiment_roberta'])/test_ns_data2.shape[0]

0.7511792452830188

In [ ]:
#graph compound true vs bert

np.sum(test_ns_data2_graph['predicted_sentiment_compound']==test_ns_data2['predicted_sentiment_bert'])/test_ns_data2.shape[0]

0.6320754716981132

In [ ]:
#graph compound true vs roberta

np.sum(test_ns_data2_graph['predicted_sentiment_compound']==test_ns_data2['predicted_sentiment_roberta'])/test_ns_data2.shape[0]

0.7488207547169812

In [ ]:
#graph bert vs roberta

np.sum(test_ns_data2['predicted_sentiment_bert']==test_ns_data2['predicted_sentiment_roberta'])/test_ns_data2.shape[0]

0.6709905660377359

# Ensembles - Comparison between two best non-GPT methods

In [ ]:
results=run_roberta_sentiment(processed_human_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at textattack/roberta-base-SST-2 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


0


In [ ]:
graph_results_by_me['Sentiment_roberta']=results
graph_results_by_4o['Sentiment_roberta']=results

In [ ]:
sentiment_merged=[ graph_results_by_me.iloc[i]['Sentiment'] if graph_results_by_me.iloc[i]['Sentiment']==graph_results_by_me.iloc[i]['Sentiment_roberta'] else 'neutral' for i in range(len(results))]
graph_results_by_me['sentiment_merged']=sentiment_merged
score_result=balanced_accuracy_score(df_my['Sentiment'], graph_results_by_me['sentiment_merged'])
print(f"Balanced accuracy for merged RoBERTa and graph based solution for both processed by human: {score_result}")

Balanced accuracy for merged RoBERTa and graph based solution for both processed by human: 0.6303875862699392


In [ ]:
score_result=balanced_accuracy_score(df_my['Sentiment'], graph_results_by_me['Sentiment_roberta'])
print(f"Balanced accuracy for RoBERTa for processed by human: {score_result}")

Balanced accuracy for RoBERTa for processed by human: 0.6122994652406417


In [ ]:
score_result=balanced_accuracy_score(df_my['Sentiment'], graph_results_by_me['Sentiment'])
print(f"Balanced accuracy for graph based solution for processed by human: {score_result}")

Balanced accuracy for graph based solution for processed by human: 0.5660679190090955


In [ ]:
score_result=balanced_accuracy_score(graph_results_by_me['Sentiment'], graph_results_by_me['Sentiment_roberta'])
print(f"Balanced accuracy between RoBERTa and graph based solution for processed by human: {score_result}")

Balanced accuracy between RoBERTa and graph based solution for processed by human: 0.4415954415954417


In [ ]:
sentiment_merged=[graph_results_by_4o.iloc[i]['Sentiment'] if graph_results_by_4o.iloc[i]['Sentiment']==graph_results_by_4o.iloc[i]['Sentiment_roberta'] else 'neutral' for i in range(len(results))]
graph_results_by_4o['sentiment_merged']=sentiment_merged
score_result=balanced_accuracy_score(df_my['Sentiment'], graph_results_by_4o['sentiment_merged'])
print(f"Balanced accuracy for merged RoBERTa and graph based solution for RoBERTa processed by human and graph by GPT 4o: {score_result}")

Balanced accuracy for merged RoBERTa and graph based solution for RoBERTa processed by human and graph by GPT 4o: 0.670208876091229


In [ ]:
score_result=balanced_accuracy_score(df_my['Sentiment'], graph_results_by_4o['Sentiment_roberta'])
print(f"Balanced accuracy for RoBERTa for processed by human: {score_result}")

Balanced accuracy for RoBERTa for processed by human: 0.6122994652406417


In [ ]:
score_result=balanced_accuracy_score(df_my['Sentiment'], graph_results_by_4o['Sentiment'])
print(f"Balanced accuracy for graph based solution for processed by human: {score_result}")

Balanced accuracy for graph based solution for processed by human: 0.6049179578591343


In [ ]:
score_result=balanced_accuracy_score(graph_results_by_4o['Sentiment'], graph_results_by_4o['Sentiment_roberta'])
print(f"Balanced accuracy between RoBERTa and graph based solution for processed by human: {score_result}")

Balanced accuracy between RoBERTa and graph based solution for processed by human: 0.44063492063492066
